In [1]:
# import and grab the data

import praw
import pandas as pd
import datetime as dt
import json


api_keys = json.loads(open('reddit_key.json').read())

reddit = praw.Reddit(
    client_id=api_keys['client_id'],
    client_secret=api_keys['client_secret'],
    user_agent="jji-bigg",
)
subreddit = reddit.subreddit("cornell")
top_subreddit = subreddit.top(limit=10)

In [78]:
def get_subreddit_data(submissions):
    topics_dict = {
        "title": [],
        "score": [],
        "id": [],
        "url": [],
        "num_comments": [],
        "created": [],
        "body": [],
    }
    for submission in submissions:
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["num_comments"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
        # print(f"submission from {submission.author}: {submission.title}")
        # print(submission.comments)
        comments_df = pd.DataFrame(columns=["comment", "score", "username", "user_id", "created"], index=None)
        # submission.comments.replace_more(limit=32)
        for comment in submission.comments.list():
            if isinstance(comment, praw.models.MoreComments):
                print(f"more comments: {comment.body}")
                continue
            # topics_dict["comment"].append(comment.body)
            # else: continue
            # print(f"comment ({comment.score}) from {comment.author}: {comment.body}\n")

            if comment.author is None:
                id = '[deleted]'
                name = '[deleted]'
            elif hasattr(comment.author, 'is_suspended') and comment.author.is_suspended:
                id = "[suspended]"
            else:
                id = comment.author.id
                name = comment.author.name

            comments_df.loc[len(comments_df)] = [
                comment.body,
                comment.score,
                name,
                id,
                comment.created,
            ]
            comments_df.sort_values(by="score", ascending=False)
        topics_dict["comments"] = comments_df.to_json(orient="split")

    return topics_dict
    # comments_df
    # topics_data = pd.DataFrame(topics_dict)

In [57]:
# print(comments_df.to_json(orient="split", indent=4, index=False))
get_subreddit_data(top_subreddit)

{'title': ['Cornell apologizing for being on Native land at graduation (they are on $8M of taxable property in the city of Ithaca and pay $96,000)'],
 'score': [1467],
 'id': ['1d2556k'],
 'url': ['https://i.redd.it/pqd332ogx13d1.png'],
 'num_comments': [136],
 'created': [1716851665.0],
 'body': [''],
 'comments': []}

# searching specific keywords in the reddit search bar
we can find the most relevant contents for each specific course; we might just perform reddit search from the LLM instead of downloading this as a dataset

In [67]:
search_4780 = subreddit.search("cs 4780", limit=10)

data = get_subreddit_data(search_4780)

In [82]:
from io import StringIO


len(data["comments"])
df = pd.read_json(StringIO(data["comments"][0]), orient="split")
df.sort_values(by="created", ascending=False)

,comment,score,username,user_id,created
31,"There is still a quiz, but it was mentioned th...",1,[deleted],[deleted],1701912296
11,The finals vary too much since there's quite a...,1,the-slow-programmer,41nuyxps,1701908889
6,Currently studying for the final -- anyone hav...,1,MoneyAd5134,9xlhbek7,1701884807
3,"i mean look at wens ""prospective note"" to peop...",2,ElevatorFantastic941,aayhu875c,1701771775
1,i think Wen Sun is a very intelligent guy and ...,8,acutegf,3f0edaug,1701689250
20,i will agree with this. unfortunately it seems...,3,acutegf,3f0edaug,1701688372
28,"take 4740 NLP instead then. Minimal proofs, a ...",1,the-slow-programmer,41nuyxps,1701674896
27,this is why Killian has a quiz to make sure pe...,1,the-slow-programmer,41nuyxps,1701674204
30,You can call something a logistical mess when ...,2,StephenBlah,nue2p5l,1701663308
29,I mean just comparing Prof Weinburgers lecture...,1,StephenBlah,nue2p5l,1701663245
